In [1]:
import scanpy as sc
import MultiMAP
import glob
import os

sc.settings.set_figure_params(dpi=80)

Load objects

In [2]:
holder = []

for file in glob.glob("/lustre/scratch126/cellgen/team205/ac65/ALI_in_vivo_raw_datasets_curated_meta/*.h5ad"):
    print(os.path.basename(file))
    holder.append(sc.read(file))
    holder[-1].obs['file_of_origin'] = os.path.basename(file)
    holder[-1].obs_names = [os.path.basename(file)+':'+i for i in holder[-1].obs_names]

Delorey_2021_COVID_only.h5ad
Yoshida_2022.h5ad
Ziegler_2021.h5ad
Bharat_2020.h5ad
Trump_2020.h5ad
Chua_2020.h5ad


/home/jovyan/my-conda-envs/d2c/lib/python3.8/site-packages/anndata/_core/anndata.py:1828: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


Loske_2021.h5ad
Melms_2021.h5ad


For whatever reason this one's cells are not named uniquely

In [3]:
holder[5].obs_names_make_unique()

These two don't have raw counts in `.X`, but they do have them in a layer

In [4]:
holder[0].X = holder[0].layers['counts']
holder[4].X = holder[4].layers['counts']

The concatenation does not want to work on these out of the box. Keep just the var and obs.

In [5]:
import anndata

for i in range(len(holder)):
    holder[i] = anndata.AnnData(X=holder[i].X, var=holder[i].var, obs=holder[i].obs)

/tmp/ipykernel_3552/4274363287.py:4: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  holder[i] = anndata.AnnData(X=holder[i].X, var=holder[i].var, obs=holder[i].obs)


Commence concatenation.

In [6]:
adata = holder[0].concatenate(holder[1:], join="inner", index_unique=None)

/home/jovyan/my-conda-envs/d2c/lib/python3.8/site-packages/anndata/_core/anndata.py:1785: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  [AnnData(sparse.csr_matrix(a.shape), obs=a.obs) for a in all_adatas],


Save the raw concatenated object for ease of access. Turn any errant `object` columns to string first so this saves.

This may not be sequentially numbered but is causal. Trying to figure out how to get this to save was fun.

In [17]:
for col in adata.obs.columns[adata.obs.dtypes == "object"]:
    adata.obs[col] = adata.obs[col].astype('str')

In [18]:
adata.write("/lustre/scratch126/cellgen/team205/kp9/jupyter/ac65/23-06-02-multimap/concatenated_raw.h5ad")